# Mapa Comparativo de Rotas V3 x V4 por Equipe usando OSRM + Folium

Este notebook:

- Lê **dois arquivos `.parquet`** de resultados para o mesmo dia e mesma equipe:
  - **V3** → ex.: `results_v3/atribuicoes_YYYY-MM-DD.parquet`
  - **V4** → ex.: `results_v4/atribuicoes_YYYY-MM-DD.parquet`
- Filtra por:
  - `dt_ref` (data de referência da simulação);
  - `equipe` (equipe específica a ser analisada).
- Ordena as OS pela hora de chegada (`dth_chegada_estimada`) em **cada versão**.
- Usa o **OSRM** para montar as rotas reais:
  - Base → OS1 → OS2 → ... → base para a V3;
  - Base → OS1 → OS2 → ... → base para a V4;
  - Construção **perna a perna** (ponto a ponto) para cada versão.
- Plota tudo no **Folium** em um único mapa:
  - **Rota V3** em uma cor (por padrão, azul) com setas indicando o sentido.
  - **Rota V4** em outra cor (por padrão, vermelho) também com setas.
  - Marcador da **base da equipe**.
  - Marcadores das OS da V3 (círculos azuis) e da V4 (círculos vermelhos), com:
    - Número da ordem de atendimento;
    - Popup indicando:
      - Versão (V3 ou V4);
      - Ordem de atendimento;
      - Equipe que atendeu;
      - Número da OS (`numos`);
      - Tipo de serviço (`tipo_serv`);
      - Horário estimado de chegada e término;
      - Tempos TD e TE.
- Opcionalmente, adiciona uma **legenda no próprio mapa** indicando o significado das cores:
  - Azul → rota e OS da versão V3;
  - Vermelho → rota e OS da versão V4.

Objetivo: **comparar visualmente o impacto das mudanças de roteirização entre V3 e V4 para uma mesma equipe e dia**, permitindo avaliar:
- Alteração da sequência de atendimento;
- Mudança no traçado da rota;
- Diferenças de cobertura espacial das OS.


In [16]:
import sys
import os
from pathlib import Path

import pandas as pd
import folium
from folium.plugins import PolyLineTextPath  # setas nas rotas
import requests


In [17]:
# Data (dt_ref) que você quer comparar PVLSN91 | 2023-03-20
DIA_REF = "2023-03-20"    # obrigatório

# Caminhos dos arquivos parquet das duas versões
PARQUET_V3_PATH = Path(f"E:/Rotas-Inteligentes/results_v3/atribuicoes_{DIA_REF}.parquet")
PARQUET_V4_PATH = Path(f"E:/Rotas-Inteligentes/results_v4/atribuicoes_{DIA_REF}.parquet")

# Base Operacional
BASE_LON = -63.885464691387746
BASE_LAT = -8.738508095069408

# Equipe que será comparada entre V3 e V4 (obrigatório)
EQUIPE_ALVO = "PVLSN91"   # ajuste para a equipe desejada

if EQUIPE_ALVO is None:
    raise ValueError("Agora é obrigatório informar uma equipe em EQUIPE_ALVO.")

# Nome do HTML de saída
OUTPUT_HTML = Path(f"mapa_comparacao_{EQUIPE_ALVO}_{DIA_REF}.html")

# ======================================================
# URL do OSRM
# ======================================================
OSRM_URL = None
try:
    # Ajuste o path abaixo se necessário para apontar para a raiz do projeto
    sys.path.append(os.getcwd())
    from v2 import config as v2_config

    OSRM_URL = v2_config.OSRM_URL.rstrip("/")
    print(f"OSRM_URL carregada de v2.config: {OSRM_URL}")
except Exception:
    # Se não conseguir importar, define manualmente:
    OSRM_URL = "http://localhost:5000"  # ajuste conforme a sua instância do OSRM
    print(f"Usando OSRM_URL manual: {OSRM_URL}")


Usando OSRM_URL manual: http://localhost:5000


In [18]:
def carregar_equipe_parquet(parquet_path: Path, dia_ref: str, equipe: str) -> pd.DataFrame:
    """
    Carrega um parquet, filtra pelo dia e pela equipe,
    e ordena pela dth_chegada_estimada.
    """
    print(f"\nCarregando parquet: {parquet_path}")
    df = pd.read_parquet(parquet_path, engine="pyarrow")
    df = pd.DataFrame(df)

    required_cols = {
        "tipo_serv",
        "numos",
        "datasol",
        "dataven",
        "datater_trab",
        "TD",
        "TE",
        "equipe",
        "dthaps_ini",
        "dthaps_fim_ajustado",
        "inicio_turno",
        "fim_turno",
        "dth_chegada_estimada",
        "dth_final_estimada",
        "fim_turno_estimado",
        "eta_source",
        "base_lon",
        "base_lat",
        "chegada_base",
        "latitude",
        "longitude",
        "dt_ref",
    }

    missing_cols = required_cols - set(df.columns)
    if missing_cols:
        raise ValueError(f"Colunas faltando no parquet {parquet_path}: {missing_cols}")

    # Normaliza dt_ref
    df["dt_ref"] = pd.to_datetime(df["dt_ref"], errors="coerce").dt.date
    dia_ref_date = pd.to_datetime(dia_ref).date()

    # Filtra pelo dia
    df = df[df["dt_ref"] == dia_ref_date].copy()
    # Filtra pela equipe
    df = df[df["equipe"] == equipe].copy()

    if df.empty:
        raise ValueError(f"Nenhuma linha encontrada para equipe '{equipe}' em {dia_ref} no arquivo {parquet_path}.")

    # Ordena pela hora de chegada
    df["dth_chegada_estimada"] = pd.to_datetime(df["dth_chegada_estimada"], errors="coerce")
    df = df.sort_values("dth_chegada_estimada").reset_index(drop=True)

    # Normaliza datas para segundos arredondados
    colsDate = [
    'datasol', 'dataven', 'datater_trab', 'dthaps_ini', 'dthaps_fim_ajustado',
    'inicio_turno', 'fim_turno', 'dthpausa_ini', 'dthpausa_fim',
    'dth_chegada_estimada', 'dth_final_estimada', 'fim_turno_estimado']

    # Converter para datetime com coerce
    df[colsDate] = df[colsDate].apply(
        lambda col: pd.to_datetime(col, errors='coerce')
    )

    # Arredondar para baixo no segundo (agora sem warning)
    df[colsDate] = df[colsDate].apply(lambda col: col.dt.floor("s"))

    # Insere coordenadas da base se não existirem
    df["base_lon"] = df["base_lon"].fillna(BASE_LON)
    df["base_lat"] = df["base_lat"].fillna(BASE_LAT)

    print(f"Linhas carregadas para equipe {equipe}: {len(df)}")
    return df


# Carrega dados da equipe para V3 e V4
df_v3 = carregar_equipe_parquet(PARQUET_V3_PATH, DIA_REF, EQUIPE_ALVO)
df_v4 = carregar_equipe_parquet(PARQUET_V4_PATH, DIA_REF, EQUIPE_ALVO)



Carregando parquet: E:\Rotas-Inteligentes\results_v3\atribuicoes_2023-03-20.parquet
Linhas carregadas para equipe PVLSN91: 15

Carregando parquet: E:\Rotas-Inteligentes\results_v4\atribuicoes_2023-03-20.parquet
Linhas carregadas para equipe PVLSN91: 15


C:\Users\junin\AppData\Local\Temp\ipykernel_2996\1007951967.py:70: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["base_lon"] = df["base_lon"].fillna(BASE_LON)
C:\Users\junin\AppData\Local\Temp\ipykernel_2996\1007951967.py:71: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["base_lat"] = df["base_lat"].fillna(BASE_LAT)


In [19]:
def osrm_route_polyline_segmentado(coords_osrm, osrm_url: str):
    """
    Recebe uma lista de coordenadas (lon, lat) e o endereço do OSRM
    e retorna uma lista de pontos (lat, lon) para desenhar a rota no Folium.

    A rota é montada perna a perna:
    (p0 -> p1), (p1 -> p2), ..., (pn-1 -> pn)
    """
    if len(coords_osrm) < 2:
        return []

    osrm_url = osrm_url.rstrip("/")
    todos_pontos = []

    for (lon1, lat1), (lon2, lat2) in zip(coords_osrm[:-1], coords_osrm[1:]):
        url = f"{osrm_url}/route/v1/driving/{lon1},{lat1};{lon2},{lat2}"
        params = {
            "overview": "full",
            "geometries": "geojson",
        }

        resp = requests.get(url, params=params, timeout=20)
        resp.raise_for_status()
        data = resp.json()

        routes = data.get("routes", [])
        if not routes:
            continue

        coords_seg = routes[0]["geometry"]["coordinates"]  # lista de [lon, lat]

        for i, (lon, lat) in enumerate(coords_seg):
            # evita repetir o primeiro ponto da perna, se já temos pontos acumulados
            if todos_pontos and i == 0:
                continue
            todos_pontos.append((lat, lon))  # Folium usa (lat, lon)

    return todos_pontos


In [20]:
def construir_mapa_comparacao(df_v3: pd.DataFrame,
                              df_v4: pd.DataFrame,
                              equipe: str,
                              osrm_url: str) -> folium.Map:
    """
    Constrói um mapa comparando a rota da equipe entre V3 e V4:
    - V3 em uma cor (ex.: azul)
    - V4 em outra cor (ex.: vermelho)
    - Ambas as rotas no mesmo mapa, com setas
    - Marcadores das OS com popup indicando versão, ordem, equipe e OS
    """

    # Escolhe base (assume que é a mesma; se não for, pega da V4 ou da V3)
    df_base = df_v4 if not df_v4.empty else df_v3
    base_lon = float(df_base["base_lon"].iloc[0])
    base_lat = float(df_base["base_lat"].iloc[0])

    # Cria mapa centralizado na base
    m = folium.Map(location=[base_lat, base_lon], zoom_start=12, tiles="OpenStreetMap")

    # Marca base
    folium.Marker(
        location=[base_lat, base_lon],
        popup=f"Base da equipe {equipe}",
        tooltip=f"Base {equipe}",
        icon=folium.Icon(color="gray", icon="home", prefix="fa"),
    ).add_to(m)

    # Função auxiliar para construir lista de coordenadas da rota (base -> OSs -> base)
    def build_coords(df):
        coords = []
        coords.append((base_lon, base_lat))
        for _, row in df.iterrows():
            try:
                lat = float(row["latitude"])
                lon = float(row["longitude"])
            except Exception:
                continue
            if pd.isna(lat) or pd.isna(lon):
                continue
            coords.append((lon, lat))
        coords.append((base_lon, base_lat))
        return coords

    coords_v3 = build_coords(df_v3)
    coords_v4 = build_coords(df_v4)

    # =======================
    # Desenhar rota V3 (azul)
    # =======================
    if len(coords_v3) >= 2:
        try:
            linha_v3 = osrm_route_polyline_segmentado(coords_v3, osrm_url)
            if linha_v3:
                rota_v3 = folium.PolyLine(
                    locations=linha_v3,
                    color="blue",
                    weight=4,
                    opacity=0.8,
                    tooltip=f"Rota V3 - equipe {equipe}",
                ).add_to(m)

                # Setas V3
                PolyLineTextPath(
                    rota_v3,
                    "➔",
                    repeat=True,
                    offset=7,
                    attributes={
                        "fill": "blue",
                        "font-weight": "bold",
                        "font-size": "12",
                    },
                ).add_to(m)
        except Exception as e:
            folium.Marker(
                location=[base_lat, base_lon],
                popup=f"Erro ao obter rota V3: {e}",
                icon=folium.Icon(color="blue", icon="exclamation-triangle", prefix="fa"),
            ).add_to(m)

    # =======================
    # Desenhar rota V4 (vermelho)
    # =======================
    if len(coords_v4) >= 2:
        try:
            linha_v4 = osrm_route_polyline_segmentado(coords_v4, osrm_url)
            if linha_v4:
                rota_v4 = folium.PolyLine(
                    locations=linha_v4,
                    color="red",
                    weight=4,
                    opacity=0.8,
                    tooltip=f"Rota V4 - equipe {equipe}",
                ).add_to(m)

                # Setas V4
                PolyLineTextPath(
                    rota_v4,
                    "➔",
                    repeat=True,
                    offset=7,
                    attributes={
                        "fill": "red",
                        "font-weight": "bold",
                        "font-size": "12",
                    },
                ).add_to(m)
        except Exception as e:
            folium.Marker(
                location=[base_lat, base_lon],
                popup=f"Erro ao obter rota V4: {e}",
                icon=folium.Icon(color="red", icon="exclamation-triangle", prefix="fa"),
            ).add_to(m)

    # =======================
    # Marcadores V3 (círculo azul)
    # =======================
    for ordem, (_, row) in enumerate(df_v3.iterrows(), start=1):
        try:
            lat = float(row["latitude"])
            lon = float(row["longitude"])
        except Exception:
            continue
        if pd.isna(lat) or pd.isna(lon):
            continue

        numos = row["numos"]
        tipo = row["tipo_serv"]
        chegada = row["dth_chegada_estimada"]
        fim = row.get("dth_final_estimada", "")
        te = row.get("TE", "")
        td = row.get("TD", "")

        popup_html = (
            f"<b>Versão:</b> V3<br>"
            f"<b>Ordem de atendimento:</b> {ordem}<br>"
            f"<b>Equipe que atendeu:</b> {equipe}<br>"
            f"<b>OS:</b> {numos}<br>"
            f"<b>Tipo:</b> {tipo}<br>"
            f"<b>Chegada estimada:</b> {chegada}<br>"
            f"<b>Fim estimado:</b> {fim}<br>"
            f"<b>TE:</b> {te} min<br>"
            f"<b>TD:</b> {td} min<br>"
        )

        folium.CircleMarker(
            location=[lat, lon],
            radius=6,
            color="blue",
            fill=True,
            fill_color="blue",
            popup=folium.Popup(popup_html, max_width=300),
            tooltip=f"V3 | Ordem {ordem} - OS {numos} ({tipo})",
        ).add_to(m)

    # =======================
    # Marcadores V4 (círculo vermelho)
    # =======================
    for ordem, (_, row) in enumerate(df_v4.iterrows(), start=1):
        try:
            lat = float(row["latitude"])
            lon = float(row["longitude"])
        except Exception:
            continue
        if pd.isna(lat) or pd.isna(lon):
            continue

        numos = row["numos"]
        tipo = row["tipo_serv"]
        chegada = row["dth_chegada_estimada"]
        fim = row.get("dth_final_estimada", "")
        te = row.get("TE", "")
        td = row.get("TD", "")

        popup_html = (
            f"<b>Versão:</b> V4<br>"
            f"<b>Ordem de atendimento:</b> {ordem}<br>"
            f"<b>Equipe que atendeu:</b> {equipe}<br>"
            f"<b>OS:</b> {numos}<br>"
            f"<b>Tipo:</b> {tipo}<br>"
            f"<b>Chegada estimada:</b> {chegada}<br>"
            f"<b>Fim estimado:</b> {fim}<br>"
            f"<b>TE:</b> {te} min<br>"
            f"<b>TD:</b> {td} min<br>"
        )

        folium.CircleMarker(
            location=[lat, lon],
            radius=6,
            color="red",
            fill=True,
            fill_color="red",
            popup=folium.Popup(popup_html, max_width=300),
            tooltip=f"V4 | Ordem {ordem} - OS {numos} ({tipo})",
        ).add_to(m)

    # Opcional: legenda simples usando um marcador fixo
    folium.map.Marker(
        [base_lat, base_lon],
        icon=folium.DivIcon(
            html=(
                '<div style="background-color: white; border: 1px solid black; '
                'padding: 4px; font-size: 10pt;">'
                '<b>Legenda:</b><br>'
                '<span style="color: blue;">&#9679;</span> Rota / OS V3<br>'
                '<span style="color: red;">&#9679;</span> Rota / OS V4<br>'
                '</div>'
            )
        ),
    ).add_to(m)

    return m


In [21]:
mapa = construir_mapa_comparacao(df_v3, df_v4, EQUIPE_ALVO, OSRM_URL)

# Salva o HTML
#mapa.save(OUTPUT_HTML)
#print(f"Mapa salvo em: {OUTPUT_HTML}")

mapa


In [ ]:
df_v4.columns


Index(['tipo_serv', 'numos', 'datasol', 'dataven', 'datater_trab', 'TD', 'TE',
       'equipe', 'dthaps_ini', 'dthaps_fim_ajustado', 'inicio_turno',
       'fim_turno', 'dthpausa_ini', 'dthpausa_fim', 'dth_chegada_estimada',
       'dth_final_estimada', 'fim_turno_estimado', 'eta_source', 'base_lon',
       'base_lat', 'chegada_base', 'latitude', 'longitude', 'dt_ref', 'EUSD',
       'EUSD_FIO_B', 'job_id_vroom'],
      dtype='object')

In [ ]:
df_v4[['dt_ref','numos','equipe','dthaps_ini', 'dthaps_fim_ajustado', 'inicio_turno',
       'fim_turno', 'dthpausa_ini', 'dthpausa_fim', 'dth_chegada_estimada',
       'dth_final_estimada', 'fim_turno_estimado', 'chegada_base', 'EUSD']]


,dt_ref,numos,equipe,dthaps_ini,dthaps_fim_ajustado,inicio_turno,fim_turno,dthpausa_ini,dthpausa_fim,dth_chegada_estimada,dth_final_estimada,fim_turno_estimado,chegada_base,EUSD
0,2023-03-20,96158873,PVLSN91,NaT,NaT,NaT,NaT,NaT,NaT,2023-03-20 21:03:19,2023-03-20 21:05:32,NaT,NaT,36.90
1,2023-03-20,96156642,PVLSN91,NaT,NaT,NaT,NaT,NaT,NaT,2023-03-20 21:08:11,2023-03-20 21:51:26,NaT,NaT,118.06
2,2023-03-20,20230000072455,PVLSN91,NaT,NaT,NaT,NaT,NaT,NaT,2023-03-20 21:55:30,2023-03-20 22:40:43,NaT,NaT,59.40
3,2023-03-20,20230000072648,PVLSN91,NaT,NaT,NaT,NaT,NaT,NaT,2023-03-20 22:42:58,2023-03-21 00:12:08,NaT,NaT,2289.62
4,2023-03-20,96158218,PVLSN91,NaT,NaT,NaT,NaT,NaT,NaT,2023-03-21 00:14:01,2023-03-21 00:23:55,NaT,NaT,151.91
5,2023-03-20,96156640,PVLSN91,NaT,NaT,NaT,NaT,NaT,NaT,2023-03-21 00:26:04,2023-03-21 00:28:05,NaT,NaT,27.55
6,2023-03-20,96157777,PVLSN91,NaT,NaT,NaT,NaT,NaT,NaT,2023-03-21 00:31:43,2023-03-21 01:02:23,NaT,NaT,118.06
7,2023-03-20,96156402,PVLSN91,NaT,NaT,NaT,NaT,NaT,NaT,2023-03-21 01:03:34,2023-03-21 01:43:44,NaT,NaT,118.06
8,2023-03-20,20230000072833,PVLSN91,NaT,NaT,NaT,NaT,NaT,NaT,2023-03-21 01:46:02,2023-03-21 02:56:33,NaT,NaT,85.68
9,2023-03-20,96157486,PVLSN91,NaT,NaT,NaT,NaT,NaT,NaT,2023-03-21 02:58:18,2023-03-21 02:58:18,NaT,NaT,12.63
